### V75 text analysis  
  
This notebook takes as input interviews of the trainers of each V75 race (avd).  
  
**Input Files**  
* Text `.txt` files with interviews.

In [1]:
!python --version

Python 3.10.12


In [2]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer # English only
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from google.colab import files
import io

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
# select file to upload
uploaded = files.upload()

Saving 20230617_V75_1.txt to 20230617_V75_1.txt


In [4]:
# check file
file_name = list(uploaded.keys())[0]
print(file_name)
print(uploaded['20230617_V75_1.txt'][0:50])
# Use utf-8 character encoding to maintain proper characters
text_full = uploaded[file_name].decode('utf-8')
print(text_full[0:50])

20230617_V75_1.txt
b'INTERVJUER V75-1 \r\n\r\nTycker han \xc3\xa4r b\xc3\xa4ttre i rygg'
INTERVJUER V75-1 

Tycker han är bättre i ryggar


In [5]:
print(len(text_full))
text_full

6678


'INTERVJUER V75-1 \r\n\r\nTycker han är bättre i ryggar\r\n1. BORUPS UMAMI - MALIN ÅBERG: Han har varit jättefin i de två senaste starterna och efter den senaste starten tog vi sikte mot det här loppet. Formen är bra och han har tränat på fint efter senaste starten. Han är startsnabb men det blir nog svårt att hålla upp ledningen och jag tycker han är bättre i ryggar men taktiken är upp till kusken. Distansen är "hans" distans och jag har goda förhoppningar om en framskjuten placering. Nu blir det första jänkarvagn sedan han kom till mig, annars fortsatt barfota runt om, norskt huvudlag och rycktussar.\r\n\r\nVår bästa chans inom V75\r\n2. GLOBAL DASH - OSKAR FLORHED: Han blir bara bättre och bättre och fortsätter att leverera varje gång. Vi trodde kanske att det skulle ta lite stopp för honom men han har verkligen gjort det bra hela året. Han åtgärdades lite efter senaste loppet och sedan har han siktats mot det här loppet. Han är inte vässad utan har gått sina normala jobb och känns 

In [16]:
# from os import listxattr

# Split code into several parts
parts: list = re.split(r'(\d+\.\s[\w\s]+-\s[\w\s]+:)', text_full)[1:]

# create empty result
result: list = []

# loop over parts
for i in range(0, len(parts), 2):
    # get nummer, namn and text
    number_name = parts[i].strip()
    number, name = number_name.split('.', 1)
    horse_name, forename_aftername = name.split('-', 1)
    text = parts[i+1].strip()

    # remove colon from name
    forename_aftername = forename_aftername.rstrip(':')

    # remove preceeding white space from text
    text = text.lstrip()

    # add results to list
    result.append([number.strip(), horse_name.strip(), forename_aftername.strip(), text])

# convert results to df
df = pd.DataFrame(result, columns=['N', 'HORSE', 'DRIVER', 'TEXT'])

# check
print(len(df))
df.head(12)

12


,N,HORSE,DRIVER,TEXT
0,1,BORUPS UMAMI,MALIN ÅBERG,Han har varit jättefin i de två senaste starte...
1,2,GLOBAL DASH,OSKAR FLORHED,Han blir bara bättre och bättre och fortsätter...
2,3,GLOBAL ATTENTION,JÖRGEN WESTHOLM,Han har fått ett fint läge eftersom han är sna...
3,4,JOY SISU,PETRI SALMELA,Hon har haft form de senaste två månaderna men...
4,5,MELLBY JOKER,OSKAR FLORHED,Han var struken för en liten febersläng för nå...
5,6,ILLFINDMYWAYHOME,DANIEL REDÉN,"Han är kraftigt på gång, man får inte stirra s..."
6,7,RARE MAGIC,JONAS EDLUND,Var jag lite besviken på direkt efteråt senast...
7,8,MAYBACH W F,MADELENE WIKSTÉN,Vi fick stryka honom till finalen senast och h...
8,9,ROLLER COASTER ÅS,CHRISTIAN MANSSON,Han gjorde flera bra lopp förra hösten. Sedan ...
9,10,JUST A FLIRT,JASMIINA AHO,Han blev skadad i ett ben i fjol och fick en p...


In [17]:
# sentiment analysis
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer # English only?

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    sentiment = sia.polarity_scores(text)['compound']
    if sentiment > 0:
        return 'positiv'
    elif sentiment < 0:
        return 'negativ'
    else:
        return 'neutral'

def get_sentiment_value(text):
  sentiment = sia.polarity_scores(text)['compound']
  return sentiment

df['SENTIMENT'] = df['TEXT'].apply(get_sentiment)
df['SENTIMENT_VALUE'] = df['TEXT'].apply(get_sentiment_value)

# Check
df.head(12)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,N,HORSE,DRIVER,TEXT,SENTIMENT,SENTIMENT_VALUE
0,1,BORUPS UMAMI,MALIN ÅBERG,Han har varit jättefin i de två senaste starte...,neutral,0.0000
1,2,GLOBAL DASH,OSKAR FLORHED,Han blir bara bättre och bättre och fortsätter...,neutral,0.0000
2,3,GLOBAL ATTENTION,JÖRGEN WESTHOLM,Han har fått ett fint läge eftersom han är sna...,neutral,0.0000
3,4,JOY SISU,PETRI SALMELA,Hon har haft form de senaste två månaderna men...,neutral,0.0000
4,5,MELLBY JOKER,OSKAR FLORHED,Han var struken för en liten febersläng för nå...,neutral,0.0000
5,6,ILLFINDMYWAYHOME,DANIEL REDÉN,"Han är kraftigt på gång, man får inte stirra s...",positiv,0.0772
6,7,RARE MAGIC,JONAS EDLUND,Var jag lite besviken på direkt efteråt senast...,positiv,0.4588
7,8,MAYBACH W F,MADELENE WIKSTÉN,Vi fick stryka honom till finalen senast och h...,neutral,0.0000
8,9,ROLLER COASTER ÅS,CHRISTIAN MANSSON,Han gjorde flera bra lopp förra hösten. Sedan ...,neutral,0.0000
9,10,JUST A FLIRT,JASMIINA AHO,Han blev skadad i ett ben i fjol och fick en p...,negativ,-0.6597


In [ ]:
# Evaluate the sentiment.
# Research methods for making sentiment analysis on Swedish text
# Or translate text to English.